# Parsing

In [1]:
import nltk
nltk.download('punkt')
nltk.download('ghostscript')
nltk.download('averaged_perceptron_tagger')
grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Error loading ghostscript: Package 'ghostscript' not found
[nltk_data]     in index
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


NLTK offers a number of different parser generators. See http://www.nltk.org/book/ch08.html for more options.

In [2]:
parser = nltk.ChartParser(grammar)

sent = nltk.word_tokenize('I shot an elephant in my pajamas')

parse_trees = list(parser.parse(sent))
for tree in parse_trees:
    print(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


## An example with recursion

In [3]:
grammar2 = nltk.CFG.fromstring("""
  S  -> NP VP
  NP -> Det Nom | PropN
  Nom -> Adj Nom | N
  VP -> V Adj | V NP | V S | V NP PP
  PP -> P NP
  PropN -> 'Buster' | 'Chatterer' | 'Joe'
  Det -> 'the' | 'a'
  N -> 'bear' | 'squirrel' | 'tree' | 'fish' | 'log'
  Adj  -> 'angry' | 'frightened' |  'little' | 'tall'
  V ->  'chased'  | 'saw' | 'said' | 'thought' | 'was' | 'put'
  P -> 'on'
  """)

In [4]:
parser = nltk.ChartParser(grammar2)

sent1 = nltk.word_tokenize('the angry bear chased the frightened little squirrel')
sent2 = nltk.word_tokenize('Chatterer said Buster thought the tree was tall')

parse_trees = list(parser.parse(sent2))
for tree in parse_trees:
    print(tree)

(S
  (NP (PropN Chatterer))
  (VP
    (V said)
    (S
      (NP (PropN Buster))
      (VP
        (V thought)
        (S (NP (Det the) (Nom (N tree))) (VP (V was) (Adj tall)))))))


# Chunking

Difference to Parsing:

* No recursion possible
* Maximum depth of parse trees
* Sentence is not required to be covered by one tree. The algorithm will find partial structures ('chunks') in the sentence.

Let us try to write a chunker for noun phrases.

In [5]:
grammar = r"""
   NP: {<DT>?(<JJ>)*<NN.*>}
"""

cp = nltk.RegexpParser(grammar)

In [6]:
tagged_tokens = nltk.pos_tag(sent1)

tree = cp.parse(tagged_tokens)
print(tree)
#tree.draw()

(S
  (NP the/DT angry/JJ bear/NN)
  chased/VBD
  (NP the/DT frightened/JJ little/JJ squirrel/NN))


In [7]:
for node in tree:
    if isinstance(node, nltk.tree.Tree):               
        if node.label() == 'NP':
            NP =  node.leaves()
            print(NP)

[('the', 'DT'), ('angry', 'JJ'), ('bear', 'NN')]
[('the', 'DT'), ('frightened', 'JJ'), ('little', 'JJ'), ('squirrel', 'NN')]


In [8]:
tagged_tokens = nltk.pos_tag(sent2)

tree = cp.parse(tagged_tokens)
print(tree)
#tree.draw()

(S
  (NP Chatterer/NNP)
  said/VBD
  (NP Buster/NNP)
  thought/VBD
  (NP the/DT tree/NN)
  was/VBD
  tall/JJ)
